In [5]:
import numpy as np
import cv2
import shutil
import sensor_msgs.point_cloud2 as pc2
import os
import rosbag
import matplotlib.pylab as plt
%matplotlib inline

# Write Camera and Lidar Frames to Disk

In [12]:
data_dir = "/media/andy/Storage/Challenge/Didi-Training-Release-1/"
bag_name = 'approach_1.bag'
bag_file = os.path.join(data_dir, bag_name)

image_dir = data_dir + bag_name.split('.')[0] + '_image/'
lidar_dir = data_dir + bag_name.split('.')[0] + '_lidar/'
#os.system('rm -r ' + image_dir)
#os.system('rm -r ' + lidar_dir)
os.system('mkdir ' + image_dir)
os.system('mkdir ' + lidar_dir)

bag = rosbag.Bag("/media/andy/Storage/Challenge/Didi-Training-Release-1/approach_1.bag", "r")
messages = bag.read_messages(topics=["/image_raw", "/velodyne_points"])

# READ AND WRITE DATA FRAMES
for msg in messages:
    #print msg.topic, msg.timestamp
    
    if 'velodyne_points' in msg.topic:
        # CONVERT MESSAGE TO A NUMPY ARRAY
        lidar = pc2.read_points(msg.message)
        lidar = np.array(list(lidar))
        
        # SAVE TO DISK
        np.save(lidar_dir + str(msg.timestamp), lidar)

    if 'image_raw' in msg.topic:
        # CONVERT MESSAGE TO A NUMPY ARRAY
        img = np.fromstring(msg.message.data, dtype=np.uint8)
        img = img.reshape(msg.message.height, msg.message.width)

        # CONVERT TO RGB
        img = cv2.cvtColor(img, cv2.COLOR_BayerGR2BGR)
        
        # SAVE TO DISK
        cv2.imwrite(image_dir + str(msg.timestamp) + '.png', img)

# Sync Camera and Lidar Frames

In [13]:
lidar_files = np.array([int(lidar.split('.')[0]) for lidar in os.listdir(lidar_dir)])
image_files = np.array([int(image.split('.')[0]) for image in os.listdir(image_dir)])

# FOR EACH IMAGE, MAKE THE NEAREST LIDAR FRAME ITS CORRESPONDENCE
for image in image_files:
    nearest_lidar = min(lidar_files, key=lambda x:abs(x-image))
    
    src_lidar = lidar_dir + str(nearest_lidar) + '.npy'
    dst_lidar = lidar_dir + str(image) + '.npy'
    
    shutil.copyfile(src_lidar, dst_lidar)

# DELETE ALL LEFTOVER LIDAR FRAMES
for lidar in lidar_files:
    if lidar not in image_files:
        redundant_lidar_file = lidar_dir + str(lidar) + '.npy'
        os.remove(redundant_lidar_file)